# Using squared-linear combination (SLC) terms with the ship loading sample

## Squared-linear combination (SLC) terms in Microsoft QIO

At the time of writing grouped terms and - more specifically - SlcTerms are in early access and only available for a subset of solvers. 

Ensure that you have the latest version of the Python SDK for Optimization - which contains the relevant functions for using SlcTerm objects - installed. 

You can use the commands below to update your SDK from inside the notebook.

In [ ]:
!pip uninstall azure-quantum -y
!pip install azure-quantum

Add your workspace details in the next cell.

You can find these details in the Azure Portal or by running `az quantum workspace show` on the command line.

In [ ]:
import time
from typing import List
from azure.quantum import Workspace
from azure.quantum.optimization import SubstochasticMonteCarlo
from azure.quantum.optimization import Problem, ProblemType, Term, SlcTerm, GroupType

workspace = Workspace (
  subscription_id = "",
  resource_group = "",
  name = "",
  location = ""
)

### The Ship Loading Sample or Freight-balancing problem (FBP)

The freight-balancing problem is a formulation of the *number partitioning problem*, which seeks a partition of a set of positive numbers into two sets that minimizes the difference between the set-wise sums. 

This example is drawn from the Microsoft Learn module [Apply QIO to a real-world problem](https://docs.microsoft.com/learn/modules/solve-quantum-inspired-optimization-problems/5-apply-quantum-inspired-optimization-real-world), where we adapt this problem for the ship-loading example. In this particular case we have a set of containers with different weights that need to be distributed between two container ships while trying to balance the load as much as possible.

Given weights (numbers) $w_1, \dots, w_d$ we seek to minimize the difference
$ \Big| \sum_{i \in A} w_i - \sum_{i \in B} w_i \Big| $
where $A$ and $B$ partition $\{1,\dots, d\}$. This is equivalent to minimizing the square $ \Big( \sum_{i \in A} w_i - \sum_{i \in B} w_i \Big)^2 . $

Hence, this Ising problem may be captured by the following cost function in factored form:
$ q(x) := \Big( \sum_{i=1}^d w_ix_i \Big)^2 , $
where $x_i = 1$ if $w_i$ is assigned to the first set $A$, while $x_i = -1$ if $w_i$ is assigned to the second set $B$.
Equivalently, we compute the expanded form:
$ q(x) = \sum_{i=1}^d w_i^2 x_i^2 + 2\sum_{i=1}^d \sum_{j=i+1}^d w_iw_jx_ix_j . $

First, we initialize a list of weights (from which we will generate the problem cost function) and instantiate a solver.

In [ ]:
from azure.quantum.target.solvers import RangeSchedule

# This array contains the weights to partition
weights = [
    2, 5, 9, 21, 35, 5, 3, 5, 10, 11,
    23, 13, 8, 7, 12, 19, 22, 54, 33,
]

# Instantiate a solver
solver = SubstochasticMonteCarlo(
    workspace,
    step_limit=10000,
    target_population=64,
    beta=RangeSchedule("linear", 0.1, 5),
    seed=42
)

#### Defining the expanded form of the cost function

We begin with the expanded form. This is the same form used in the [ship loading sample](../ship-loading/README.md) and the [Microsoft Learn module](https://docs.microsoft.com/learn/modules/solve-quantum-inspired-optimization-problems/5-apply-quantum-inspired-optimization-real-world).

Since Ising problems bear spin variables with values $\pm 1$, we may simplify the expression to:
$$ q(x) = \sum_{i=1}^d w_i^2 + 2\sum_{i=1}^d \sum_{j=i+1}^d w_iw_jx_ix_j $$
When creating the `Problem`, we will exclude the constant terms since they are unaffected by optimization. 

To interpret the cost returned by the Azure Quantum solver, we will need to add these back in.
You can see how this is done at the end of this sample.

In [ ]:
def createFBP_expanded(weights: List[int]) -> Problem:
    # Expand the squared summation
    terms = []
    for i in range(len(weights)):
        for j in range(i+1, len(weights)):
            terms.append(
                Term(
                    c = 2 * weights[i] * weights[j],
                    indices = [i, j]
                )
            )

    # Return an Ising-type problem
    return Problem(name="Freight Balancing Problem", problem_type=ProblemType.ising, terms=terms)

# Create expanded problem for the given list of weights:
problem_expanded = createFBP_expanded(weights)

#### Defining the factored form of the cost function

With the factored form, we may simply represent the cost function in a form more closely resembling the original derivation, recall:
$$ q(x) = \Big( \sum_{i=1}^d w_ix_i \Big)^2 . $$

In [ ]:
def createFBP_factored(weights: List[int]) -> Problem:
    # Construct the factored form
    terms = [
        SlcTerm(
            c = 1,
            terms = [
                Term(
                    c = weights[i],
                    indices = [i]
                )
            for i in range(len(weights))]
        )
    ]
    
    # Return an Ising-type problem
    return Problem(name="Freight Balancing Problem", problem_type=ProblemType.ising, terms=terms)

# Create expanded problem for the given list of weights:
problem_factored = createFBP_factored(weights)

#### Running both problem formulations

As a next step we will run both problem formulations and compare the runtime on the service.

To make sure that we've achieved the same result quality we need to add the constant cost to the cost value of the result of the expanded problem formulation. 

In this case we should end up with -7860 and 1 as cost values for the expanded and the factored form respectively. 

The constant cost should be 7861 which means that both solutions have essentially the same cost.

In [ ]:
# Optimize the expanded problem
print('Submitting expanded problem...')
start = time.time()
result = solver.optimize(problem_expanded)
timeElapsed = time.time() - start
print('Result in {:.1f} seconds: '.format(timeElapsed), result)

# To interpret reported cost, we must add in the constant value from w_i^2x_i^2 terms
constant_cost = 0
for w in weights:
    constant_cost += w**2
print('constant cost: ', constant_cost)

In [ ]:
# Optimize the factored problem
print('Submitting factored problem...')
start = time.time()
result = solver.optimize(problem_factored)
timeElapsed = time.time() - start
print('Result in {:.1f} seconds: '.format(timeElapsed), result)